In [5]:
# Cell 1: Setup and Imports

# --- Package Installation ---
# Install necessary packages with specific versions for compatibility
%pip install tensorflow==2.15.0 keras==2.15.0 protobuf==4.25.3
%pip install yolov5 pandas opencv-python torch torchvision tqdm matplotlib Pillow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# --- Library Imports ---
import os
import shutil
import cv2
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from tqdm.notebook import tqdm
from IPython.display import display, HTML
import matplotlib.pyplot as plt # <-- LÍNEA IMPORTANTE AÑADIDA
import ipywidgets


# --- Environment Configuration ---
# Suppress verbose logs for a cleaner output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import warnings
warnings.filterwarnings('ignore')

print("Setup complete. All libraries are installed and imported.")


Setup complete. All libraries are installed and imported.


In [ ]:
# Cell 2: Model Loading

# --- Configuration ---
YOLO_MODEL_PATH = 'LandDenoisedAugmented.pt'

YOLO_REPO_PATH = 'yolov5' # Assumes yolov5 repo is cloned
LAND_DATASET_IMAGES = 'HRSID_land_main/images/val'
LAND_DATASET_LABELS = 'HRSID_land_main/labels/val'
SEA_DATASET_IMAGES = 'sea_dataset/images/'
SEA_DATASET_LABELS = 'sea_dataset/labels/'
DENOISEDLAND_DATASET_IMAGES = 'val_denoised'

# --- Load YOLOv5 Detection Model ---
print(f"Loading YOLOv5 model from '{YOLO_MODEL_PATH}'...")
try:
    yolo_model = torch.hub.load(YOLO_REPO_PATH, 'custom', path=YOLO_MODEL_PATH, source='local', force_reload=True)
    if torch.cuda.is_available():
        yolo_model.to('cuda')
    print("✅ YOLOv5 model loaded successfully.")
except Exception as e:
    print(f"❌ Error loading YOLOv5 model: {e}")


Loading YOLOv5 model from 'LandDenoisedAugmented.pt'...
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\angel/.cache\torch\hub\master.zip


YOLOv5  v7.0-422-g2540fd4c Python-3.10.18 torch-2.7.1+cpu CPU



❌ Error loading YOLOv5 model: cannot instantiate 'PosixPath' on your system. Cache may be out of date, try `force_reload=True` or see https://docs.ultralytics.com/yolov5/tutorials/pytorch_hub_model_loading for help.


In [12]:
# Cell 3: Helper Functions (CORRECTED AND IMPROVED)

def calculate_iou(boxA, boxB):
    """
    Calculates the Intersection over Union (IoU) between two bounding boxes.
    THIS FUNCTION IS NOW CORRECTED.
    """
    # Determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # Compute the area of intersection
    interArea = max(0, xB - xA) * max(0, yB - yA)

    # Compute the area of both the prediction and ground-truth rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # Compute the area of the union
    unionArea = float(boxAArea + boxBArea - interArea)

    # Compute the IoU
    iou = interArea / unionArea if unionArea > 0 else 0
    return iou

def load_yolo_labels(label_path, img_width, img_height):
    """Loads ground truth boxes from a YOLO .txt file and denormalizes them."""
    boxes = []
    if not os.path.exists(label_path): return boxes
    with open(label_path, 'r') as f:
        for line in f:
            _, x_center, y_center, width, height = map(float, line.strip().split())
            w_abs, h_abs = width * img_width, height * img_height
            xmin = (x_center * img_width) - (w_abs / 2)
            ymin = (y_center * img_height) - (h_abs / 2)
            boxes.append([xmin, ymin, xmin + w_abs, ymin + h_abs])
    return boxes

def evaluate_performance(predictions, ground_truth, iou_threshold=0.1):
    """
    Evaluates detections against ground truth to calculate TP, FP, FN.
    This logic has been slightly improved for clarity.
    """
    matches = []
    
    # Use copies to safely remove matched items
    gt_boxes_unmatched = list(ground_truth)
    pred_boxes_unmatched = list(predictions)

    tp = 0
    
    # First, find all True Positives by matching predictions to ground truth
    for pred_box in list(pred_boxes_unmatched): # Iterate over a copy
        best_iou = 0
        best_gt_match = None
        
        for gt_box in gt_boxes_unmatched:
            iou = calculate_iou(pred_box, gt_box)
            if iou > best_iou:
                best_iou = iou
                best_gt_match = gt_box
        
        if best_iou > iou_threshold:
            tp += 1
            matches.append({'box': pred_box, 'status': 'TP'})
            # Remove both matched boxes so they can't be used again
            pred_boxes_unmatched.remove(pred_box)
            gt_boxes_unmatched.remove(best_gt_match)

    # Whatever is left are False Positives and False Negatives
    fp = len(pred_boxes_unmatched)
    for pred_box in pred_boxes_unmatched:
        matches.append({'box': pred_box, 'status': 'FP'})
        
    fn = len(gt_boxes_unmatched)
    for gt_box in gt_boxes_unmatched:
        matches.append({'box': gt_box, 'status': 'FN'})

    return {'tp': tp, 'fp': fp, 'fn': fn}, matches

def draw_results_on_image(image, matches):
    """Draws colored bounding boxes on an image based on match status."""
    color_map = {
        'TP': (0, 255, 0),   # Green
        'FP': (0, 0, 255),   # Red
        'FN': (255, 0, 0)    # Blue
    }
    for match in matches:
        xmin, ymin, xmax, ymax = map(int, match['box'])
        status = match['status']
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color_map[status], 2)
    return image

print("Helper functions defined (with corrected evaluation method).")

Helper functions defined (with corrected evaluation method).


In [13]:
# Final Cell: Complete Analysis of the Validation Dataset

import io
import shutil
import pandas as pd
from IPython.display import display, HTML
from tqdm.notebook import tqdm
import ipywidgets as widgets

def run_final_analysis(image_dir, label_dir, model, output_dir):
    """
    Processes all images in a dataset, saves the visual results to a folder,
    and displays the final aggregated performance metrics.
    """
    # --- STAGE 1: Setup and Directory Preparation ---
    display(HTML(f"<h3>Starting Full Dataset Analysis</h3>"))
    
    if os.path.exists(output_dir):
        print(f"Cleaning up previous results directory: '{output_dir}'")
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)
    print(f"Results folder created at: '{output_dir}'")
    
    # Check if the image directory exists
    if not os.path.isdir(image_dir):
        print(f"❌ Error: Image directory not found at '{image_dir}'")
        return

    image_files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    
    # Initialize counters
    yolo_totals = {'tp': 0, 'fp': 0, 'fn': 0}

    # --- STAGE 2: Process all images ---
    for image_name in tqdm(image_files, desc="Analyzing all images"):
        image_path = os.path.join(image_dir, image_name)
        label_path = os.path.join(label_dir, os.path.splitext(image_name)[0] + '.txt')
        
        # Load data
        original_image_bgr = cv2.imread(image_path)
        original_image_rgb = cv2.cvtColor(original_image_bgr, cv2.COLOR_BGR2RGB)
        h, w, _ = original_image_bgr.shape
        
        gt_boxes = load_yolo_labels(label_path, w, h)
        
        # Get YOLO predictions
        yolo_results = model(original_image_rgb)
        yolo_predictions = [pred[:4].tolist() for pred in yolo_results.xyxy[0].cpu().numpy()]

        # Evaluate performance for this image
        performance, matches = evaluate_performance(yolo_predictions, gt_boxes)
        
        # Aggregate totals
        for key in yolo_totals:
            yolo_totals[key] += performance[key]
        
        # Draw results on the image
        image_with_boxes = draw_results_on_image(original_image_bgr.copy(), matches)
        
        # Save the resulting image to the output directory
        output_image_path = os.path.join(output_dir, image_name)
        cv2.imwrite(output_image_path, image_with_boxes)

    # --- STAGE 3: Display all analytics in the notebook ---
    
    display(HTML(f"<hr><h2>Final Analysis Results</h2>"))
    display(HTML(f"<p>A total of <b>{len(image_files)}</b> images have been processed. The visual results have been saved to the <b>'{output_dir}'</b> folder.</p>"))

    # 1. Summary Table (TP, FP, FN)
    df_data = {
        'Metric': ['True Positives (TP)', 'False Positives (FP)', 'False Negatives (FN)'],
        'Total Results': [yolo_totals['tp'], yolo_totals['fp'], yolo_totals['fn']]
    }
    df_summary = pd.DataFrame(df_data).set_index('Metric')
    display(HTML("<h4>Aggregated Confusion Matrix</h4>"))
    display(df_summary)

    # 2. Performance Metrics (Precision, Recall, F1-Score)
    tp, fp, fn = yolo_totals['tp'], yolo_totals['fp'], yolo_totals['fn']
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    display(HTML(f"""
    <div style="border: 1px solid #ccc; padding: 10px; margin-top: 20px;">
        <h4>Performance Metrics</h4>
        <p>
            <b>Precision:</b> {precision:.2%} (Of all detections made, this % was correct)<br>
            <b>Recall (Sensitivity):</b> {recall:.2%} (Of all actual targets, this % was found)<br>
            <b>F1-Score:</b> {f1_score:.3f} (The balance between Precision and Recall)
        </p>
    </div>
    """))

    # 3. Results Graph
    fig, ax = plt.subplots(figsize=(8, 5))
    metrics = ['True Positives', 'False Positives', 'False Negatives']
    counts = [tp, fp, fn]
    colors = ['green', 'red', 'blue']
    ax.bar(metrics, counts, color=colors)
    ax.set_ylabel('Total Count')
    ax.set_title('Model Results Summary')
    for i, v in enumerate(counts):
        ax.text(i, v + (max(counts) * 0.01), str(v), ha='center', fontweight='bold')
    
    # Save plot to buffer and display as a widget
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    buf.seek(0)
    graph_widget = ipywidgets.Image(value=buf.read(), format='png')
    display(HTML("<h4>Results Chart</h4>"))
    display(graph_widget)
    buf.close()
    plt.close(fig)

In [14]:
# Cell to Find the Optimal YOLOv5 Confidence Threshold

import io
import matplotlib.pyplot as plt
import ipywidgets as widgets
import pandas as pd
import numpy as np
import os
from PIL import Image
from tqdm.notebook import tqdm
from IPython.display import display, HTML

def find_yolo_optimal_threshold(image_dir, label_dir, model, num_images_to_test=None):
    """
    Analyzes YOLOv5 performance across a range of confidence thresholds
    to find the optimal value.
    """
    display(HTML(f"<h3>Analyzing the Optimal Confidence Threshold for YOLOv5</h3>"))
    
    # --- STAGE 1: Setup ---
    if not os.path.isdir(image_dir):
        print(f"❌ Error: Image directory not found at '{image_dir}'")
        return

    # Get the list of images to process
    image_files_all = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    if num_images_to_test:
        image_files = image_files_all[:num_images_to_test]
    else:
        image_files = image_files_all
        
    print(f"{len(image_files)} images will be analyzed for each threshold. This may take a while...")
    
    # --- STAGE 2: Loop through thresholds and evaluate ---
    thresholds_to_test = np.arange(0.10, 0.95, 0.05) # Test from 10% to 90%
    analysis_results = []
    original_conf = model.conf # Save original confidence to restore it later

    for threshold in tqdm(thresholds_to_test, desc="Testing Thresholds"):
        # Set the model's confidence threshold for this run
        model.conf = threshold
        
        # Reset counters for this threshold
        total_tp, total_fp, total_fn = 0, 0, 0
        
        # Inner loop to process all images with the new threshold
        for image_name in image_files:
            image_path = os.path.join(image_dir, image_name)
            label_path = os.path.join(label_dir, os.path.splitext(image_name)[0] + '.txt')

            # Load data
            original_image_rgb = np.array(Image.open(image_path).convert("RGB"))
            h, w, _ = original_image_rgb.shape
            gt_boxes = load_yolo_labels(label_path, w, h)
            
            # Run inference (will now use the new model.conf)
            yolo_results = model(original_image_rgb)
            yolo_predictions = [pred[:4].tolist() for pred in yolo_results.xyxy[0].cpu().numpy()]
            
            # Evaluate performance
            performance, _ = evaluate_performance(yolo_predictions, gt_boxes)
            total_tp += performance['tp']
            total_fp += performance['fp']
            total_fn += performance['fn']
            
        # Calculate final metrics for this threshold
        precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
        recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        analysis_results.append({
            "Threshold": f"{threshold:.2f}",
            "TP": total_tp, "FP": total_fp, "FN": total_fn,
            "Precision": precision, "Recall": recall, "F1-Score": f1_score
        })

    # --- STAGE 3: Display Results ---
    model.conf = original_conf # Restore the model's original confidence
    
    df_results = pd.DataFrame(analysis_results)
    
    if not df_results.empty:
        df_results['F1-Score'] = pd.to_numeric(df_results['F1-Score'])
        best_f1_score = df_results['F1-Score'].max()
        
        styled_df = df_results.style.apply(lambda s: ['background-color: yellow' if v == best_f1_score else '' for v in s], subset=['F1-Score'])\
                                      .format({"Precision": "{:.2%}", "Recall": "{:.2%}", "F1-Score": "{:.3f}"})
        
        display(HTML("<h4>YOLOv5 Performance vs. Confidence Threshold</h4>"))
        display(styled_df)

        # Create and display the graph
        fig, ax = plt.subplots(figsize=(12, 7))
        ax.plot(df_results["Threshold"], df_results["Precision"], 'b-o', label='Precision')
        ax.plot(df_results["Threshold"], df_results["Recall"], 'g-o', label='Recall')
        ax.plot(df_results["Threshold"], df_results["F1-Score"], 'r-s', label='F1-Score', linewidth=3)
        
        best_f1_row = df_results.loc[df_results['F1-Score'].idxmax()]
        ax.axvline(x=best_f1_row['Threshold'], color='grey', linestyle='--', label=f'Best F1-Score ({best_f1_row["F1-Score"]:.3f}) at Threshold {best_f1_row["Threshold"]}')
        
        ax.set_title('YOLOv5 Model Performance vs. Confidence Threshold')
        ax.set_xlabel('Detection Confidence Threshold')
        ax.set_ylabel('Score')
        ax.grid(True)
        ax.legend()
        
        buf = io.BytesIO()
        fig.savefig(buf, format='png', bbox_inches='tight')
        buf.seek(0)
        graph_widget = ipywidgets.Image(value=buf.read(), format='png')
        display(graph_widget)
        buf.close()
        plt.close(fig)

# Analysis on `land_dataset`

In [15]:
# --- Run the analysis function using the paths defined in Cell 2 ---
# (Asegúrate de que las variables LAND_DATASET_IMAGES y LAND_DATASET_LABELS existen)
if 'LAND_DATASET_IMAGES' in locals() and 'LAND_DATASET_LABELS' in locals():
    run_final_analysis(
        image_dir=LAND_DATASET_IMAGES,
        label_dir=LAND_DATASET_LABELS,
        model=yolo_model,
        output_dir = 'landresults'
    )
else:
    print("❌ Error: Las variables 'LAND_DATASET_IMAGES' y 'LAND_DATASET_LABELS' no están definidas.")
    print("Por favor, ejecuta la Celda 2 para definirlas antes de ejecutar esta celda.")

Cleaning up previous results directory: 'landresults'
Results folder created at: 'landresults'


Analyzing all images:   0%|          | 0/169 [00:00<?, ?it/s]

,Total Results
Metric,
True Positives (TP),942
False Positives (FP),300
False Negatives (FN),287


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\xbb\x00\x00\x01\xc4\x08\x06\x00\x00\x009\xff\xe2…

In [16]:
# --- Run the analysis function using the paths defined in Cell 2 ---
# (Asegúrate de que las variables LAND_DATASET_IMAGES y LAND_DATASET_LABELS existen)
if 'LAND_DATASET_IMAGES' in locals() and 'LAND_DATASET_LABELS' in locals():
    # Puedes limitar el número de imágenes para una prueba más rápida,
    # o poner None para analizar todo el dataset.
    find_yolo_optimal_threshold(
        image_dir=LAND_DATASET_IMAGES,
        label_dir=LAND_DATASET_LABELS,
        model=yolo_model,
        num_images_to_test=100 # Analizar 100 imágenes para un resultado rápido y representativo
    )
else:
    print("❌ Error: Las variables 'LAND_DATASET_IMAGES' y 'LAND_DATASET_LABELS' no están definidas.")
    print("Por favor, ejecuta la Celda 2 para definirlas antes de ejecutar esta celda.")

100 images will be analyzed for each threshold. This may take a while...


Testing Thresholds:   0%|          | 0/17 [00:00<?, ?it/s]

,Threshold,TP,FP,FN,Precision,Recall,F1-Score
0,0.10,371,291,62,56.04%,85.68%,0.678
1,0.15,352,187,81,65.31%,81.29%,0.724
2,0.20,339,138,94,71.07%,78.29%,0.745
3,0.25,335,105,98,76.14%,77.37%,0.767
4,0.30,324,84,109,79.41%,74.83%,0.771
5,0.35,315,67,118,82.46%,72.75%,0.773
6,0.40,310,54,123,85.16%,71.59%,0.778
7,0.45,299,36,134,89.25%,69.05%,0.779
8,0.50,288,28,145,91.14%,66.51%,0.769
9,0.55,281,23,152,92.43%,64.90%,0.763


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\xec\x00\x00\x02r\x08\x06\x00\x00\x00^\xc2J\x0f\x…

# Analysis in "sea_dataset"

In [17]:
# --- Run the analysis function using the paths defined in Cell 2 ---
# (Asegúrate de que las variables LAND_DATASET_IMAGES y LAND_DATASET_LABELS existen)
if 'LAND_DATASET_IMAGES' in locals() and 'LAND_DATASET_LABELS' in locals():
    run_final_analysis(
        image_dir=SEA_DATASET_IMAGES,
        label_dir=SEA_DATASET_LABELS,
        model=yolo_model,
        output_dir = 'searesults'
    )
else:
    print("❌ Error: Las variables 'LAND_DATASET_IMAGES' y 'LAND_DATASET_LABELS' no están definidas.")
    print("Por favor, ejecuta la Celda 2 para definirlas antes de ejecutar esta celda.")

Results folder created at: 'searesults'


Analyzing all images:   0%|          | 0/246 [00:00<?, ?it/s]

,Total Results
Metric,
True Positives (TP),329
False Positives (FP),7
False Negatives (FN),5


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\xbb\x00\x00\x01\xc4\x08\x06\x00\x00\x009\xff\xe2…

In [18]:
# --- Run the analysis function using the paths defined in Cell 2 ---
find_yolo_optimal_threshold(
        image_dir=SEA_DATASET_IMAGES,
        label_dir=SEA_DATASET_LABELS,
        model=yolo_model,
        num_images_to_test=100 # Analizar 100 imágenes para un resultado rápido y representativo
    )

100 images will be analyzed for each threshold. This may take a while...


Testing Thresholds:   0%|          | 0/17 [00:00<?, ?it/s]

,Threshold,TP,FP,FN,Precision,Recall,F1-Score
0,0.10,142,12,1,92.21%,99.30%,0.956
1,0.15,142,7,1,95.30%,99.30%,0.973
2,0.20,142,6,1,95.95%,99.30%,0.976
3,0.25,141,5,2,96.58%,98.60%,0.976
4,0.30,140,4,3,97.22%,97.90%,0.976
5,0.35,139,4,4,97.20%,97.20%,0.972
6,0.40,138,3,5,97.87%,96.50%,0.972
7,0.45,137,2,6,98.56%,95.80%,0.972
8,0.50,135,2,8,98.54%,94.41%,0.964
9,0.55,135,1,8,99.26%,94.41%,0.968


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\xec\x00\x00\x02r\x08\x06\x00\x00\x00^\xc2J\x0f\x…

## Denoised Dataset

In [19]:
# --- Run the analysis function using the paths defined in Cell 2 ---
# (Asegúrate de que las variables LAND_DATASET_IMAGES y LAND_DATASET_LABELS existen)
if 'LAND_DATASET_IMAGES' in locals() and 'LAND_DATASET_LABELS' in locals():
    run_final_analysis(
        image_dir=DENOISEDLAND_DATASET_IMAGES,
        label_dir=LAND_DATASET_LABELS,
        model=yolo_model,
        output_dir = 'Denoised_Land_Results'
    )
else:
    print("❌ Error: Las variables 'LAND_DATASET_IMAGES' y 'LAND_DATASET_LABELS' no están definidas.")
    print("Por favor, ejecuta la Celda 2 para definirlas antes de ejecutar esta celda.")

Results folder created at: 'Denoised_Land_Results'


Analyzing all images:   0%|          | 0/169 [00:00<?, ?it/s]

,Total Results
Metric,
True Positives (TP),1008
False Positives (FP),462
False Negatives (FN),221


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\xc4\x00\x00\x01\xc4\x08\x06\x00\x00\x00>\xbd<\x1…

In [20]:
# --- Run the analysis function using the paths defined in Cell 2 ---
# (Asegúrate de que las variables LAND_DATASET_IMAGES y LAND_DATASET_LABELS existen)
if 'LAND_DATASET_IMAGES' in locals() and 'LAND_DATASET_LABELS' in locals():
    # Puedes limitar el número de imágenes para una prueba más rápida,
    # o poner None para analizar todo el dataset.
    find_yolo_optimal_threshold(
        image_dir=DENOISEDLAND_DATASET_IMAGES,
        label_dir=LAND_DATASET_LABELS,
        model=yolo_model,
        num_images_to_test=100 # Analizar 100 imágenes para un resultado rápido y representativo
    )
else:
    print("❌ Error: Las variables 'LAND_DATASET_IMAGES' y 'LAND_DATASET_LABELS' no están definidas.")
    print("Por favor, ejecuta la Celda 2 para definirlas antes de ejecutar esta celda.")

100 images will be analyzed for each threshold. This may take a while...


Testing Thresholds:   0%|          | 0/17 [00:00<?, ?it/s]

,Threshold,TP,FP,FN,Precision,Recall,F1-Score
0,0.10,386,359,47,51.81%,89.15%,0.655
1,0.15,374,239,59,61.01%,86.37%,0.715
2,0.20,359,191,74,65.27%,82.91%,0.730
3,0.25,351,157,82,69.09%,81.06%,0.746
4,0.30,348,117,85,74.84%,80.37%,0.775
5,0.35,340,97,93,77.80%,78.52%,0.782
6,0.40,327,78,106,80.74%,75.52%,0.780
7,0.45,317,67,116,82.55%,73.21%,0.776
8,0.50,308,50,125,86.03%,71.13%,0.779
9,0.55,300,35,133,89.55%,69.28%,0.781


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\xec\x00\x00\x02r\x08\x06\x00\x00\x00^\xc2J\x0f\x…